In [ ]:
def data_pred(data,start_date,end_date,split_date):
    training_period=customer_ds.loc[(customer_ds.InvoiceDate >= start_date) & (customer_ds.InvoiceDate <= split_date)]
    testing_period=customer_ds.loc[(customer_ds.InvoiceDate > split_date) & (customer_ds.InvoiceDate <= end_date)]
    data=pd.DataFrame(training_period['CustomerID'].unique())
    data.columns = ['CustomerID']
    #first purchase in testing period
    testing_period_first_purchase = testing_period.groupby('CustomerID').InvoiceDate.min().reset_index()
    testing_period_first_purchase.columns = ['CustomerID','MinPurchaseDate']

    #Last purchase date in training period
    training_period_last_purchase = training_period.groupby('CustomerID').InvoiceDate.max().reset_index()
    training_period_last_purchase.columns = ['CustomerID','MaxPurchaseDate']
    train_test_merge=pd.merge(training_period_last_purchase,testing_period_first_purchase,on='CustomerID',how='left')
    #n+1 - n
    train_test_merge['NextPurchaseDay_TestPeriod'] = (train_test_merge['MinPurchaseDate'] - train_test_merge['MaxPurchaseDate']).dt.days
    data=pd.merge(data, train_test_merge[['CustomerID','NextPurchaseDay_TestPeriod']],on='CustomerID',how='left')
    #NaN means they do not have a purchase date in that period . 
    #for analysis we could consider it as a large next purchase date and fill that number by 999
    data = data.fillna(999)
    df_Invoice_Day = training_period[['CustomerID','InvoiceDate']]
    df_Invoice_Day['InvoiceDay'] = training_period['InvoiceDate'].dt.date
    df_Invoice_Day = df_Invoice_Day.sort_values(['CustomerID','InvoiceDate'])
    df_Invoice_Day = df_Invoice_Day.drop_duplicates(subset=['CustomerID','InvoiceDay'],keep='first')
    #shifting last 3 purchase dates
    df_Invoice_Day['n-1'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(1)
    df_Invoice_Day['n-2'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(2)
    df_Invoice_Day['n-3'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(3)
    df_Invoice_Day['DayDiff1'] = (df_Invoice_Day['InvoiceDay'] - df_Invoice_Day['n-1']).dt.days
    df_Invoice_Day['DayDiff2'] = (df_Invoice_Day['n-1'] - df_Invoice_Day['n-2']).dt.days
    df_Invoice_Day['DayDiff3'] = (df_Invoice_Day['n-2'] - df_Invoice_Day['n-3']).dt.days
    day_order_last = df_Invoice_Day.drop_duplicates(subset=['CustomerID'],keep='last')
    day_order_last = day_order_last.dropna()
    data = pd.merge(data, day_order_last[['CustomerID','InvoiceDay','DayDiff1','DayDiff2','DayDiff3']], on='CustomerID')
    data['Average']=(data['DayDiff1']+data['DayDiff2']+data['DayDiff3'])/3
    data['NextPurchaseDayRange'] = 4
    data.loc[data.NextPurchaseDay_TestPeriod<20,'NextPurchaseDayRange'] = 0
    data.loc[data.NextPurchaseDay_TestPeriod<50,'NextPurchaseDayRange'] = 1
    data.loc[data.NextPurchaseDay_TestPeriod<100,'NextPurchaseDayRange'] = 2
    data.loc[data.NextPurchaseDay_TestPeriod<150,'NextPurchaseDayRange'] = 3
    return data
 